# Role of *footprints* in the creation of objects 

Let's take the previous example of creating a *resource* object: 

```python
pe_resource = fp.proxy.resource(kind='gridpoint',
                                term=3,
                                geometry='euroc25',
                                nativefmt='grib',
                                model='arpege',
                                cutoff='production',
                                date='2017060118',
                                origin='historic')
```

We call generic a method that creates the “appropriate” object based on the attributes passed to it: this remind us of the factory design pattern. This is one way to describe the role of the **footprints** package, nonetheless **footprints** do much more than to implement a factory (in particular in terms of flexibility).

## Description of objects managed by _footprints_

We will now start our demonstration of the _footprints_ package; the least is to load the package: 

In [1]:
import footprints as fp

In itself, loading the **footprints** package does nothing. Everything is decided when writing the targerted classes and creating the objects. 

### Classes, Collectors and Objects

Here is an example of class backed to the _footprints_ package:

In [2]:
class Observations(fp.FootprintBase):
    
    _collector = ('fakeresource', )
    _footprint = dict(
        info = 'Any kind of insitu observations',
        attr = dict(
            kind = dict(
                values = ['observations', ],
                info   = 'The kind of resource',
            )
        )
    )

* Any “footprinted” class must inherit from **FootprintBase**
* A “footprinted” class is attached to one (or more) collector. Each collector is an object of class **footprints.collectors.Collector** which maintains, for a given theme (here “fakeresource”), a list of the different classes that have been declared. 
* Each “footprinted” class must have a footprint that specifies the attributes of the class, ...

The retrieval of a collector object can be done via the **collectors** module: 

In [3]:
print(fp.collectors.get(tag='fakeresource'))

Usualy, another syntax is preferred: 

In [4]:
from footprints import proxy as fpx
print(fpx.fakeresources)  # Watch out for "s" at the end

What are the classes attached to the “fakeresource” collector? 

In [5]:
print([r for r in fpx.fakeresources])

[<class '__main__.Observations'>]


We find again the class that we defined previously... 

As a catalog of the different available classes, the _collector_ allows the creation of objects: 

In [6]:
obs_obj1 = fpx.fakeresources.load(kind='observations')
print(obs_obj1)

<__main__.Observations object at 0x7fbcdfb2ec88 | footprint=1>


Again, the **footprints** package provides a simplified syntax that is often preferred over the previous one: 

In [9]:
obs_obj2 = fpx.fakeresource(kind='observations')  # Beware: no "s" at the end
print(obs_obj2)

<__main__.Observations object at 0x7fbcdfb2ecf8 | footprint=1>


From this simple example we can glimpse the possibility of errors... What happens if we ask for an object whose *kind* does not exist? 

In [10]:
failer1 = fpx.fakeresource(kind='foo')
print('failer1 is:', str(failer1))

# [2019/08/27-16:36:04][footprints.collectors][pickup_and_cache:0296][WARNING]: No fakeresource found in description 
      dict(
          fakeresource = None, 
          kind = 'foo',
      )



 Report Footprint-Fakeresource: 

     __main__.Observations
         kind       : {'args': 'foo', 'why': 'Not in values'}

failer1 is: None


In case of error, the *collector* returns *None* and an error message is displayed detailing why it is not possible to instantiate this or that class.

### Inheritance and predefined footprints 

In a language like Python, the use of class inheritance is more than recommended. Therefore, the **footprints** package must handle this in an optimal way. In practice, any class inheriting a “footprinted” class also inherits the footprint of the parent class. This footprint can then be altered or extended to the level of the child class. 

**footprints** also offers the possibility to create abstract classes. This makes it possible to define classes that are essential to the structure of the inheritance tree, while being certain that these will never be instantiated. 

Some attributes are “classic”, we can imagine to predefine them. 

In a numerical prediction context, we can for example consider that the *cutoff* is a very common attribute (assimilation or production) and want to predefine it: 

In [11]:
a_cutoff = dict(
    info     = "The cutoff type of the generating process.",
    values   = ['assim', 'production', ],
)
cutoff = fp.Footprint(info = 'Abstract cutoff', attr = dict(cutoff = a_cutoff))

This predefined attribute can then be used to define an abstract class that serves as a base for different model fields:

In [12]:
class ModelState(fp.FootprintBase):

    _collector = ('fakeresource', )
    _abstract = True
    _footprint = [
        cutoff,
        dict(
            info = 'Abstract Model State',
            attr = dict(
                kind = dict(
                    info = 'The kind of modelstates we are dealing with',
                ),
                date = dict(
                    info = 'The base date of this model state'
                ),
            )
        )
    ]

Let's inherit from this class to create initial conditions:

In [13]:
class InitialConditions(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['initial_conditions', ]
            ),       
        )
    )    

In [14]:
ic_obj = fpx.fakeresource(kind='initial_conditions', date='2017070100', cutoff='assim')
print(ic_obj)
print('kind={0.kind:s}, date={0.date:s}, cutoff={0.cutoff:s}'.format(ic_obj))

<__main__.InitialConditions object at 0x7fbcdfb3e8d0 | footprint=3>
kind=initial_conditions, date=2017070100, cutoff=assim


Predefined footprints are very useful for defining attributes, but one would want to go further and alter the inner-working of the class (because of the addition of this attribute). Practical example: 

  * Add a *cutoff* attribute (this is what we did before)
  * Add to the class a property that returns a shortened version of the cutoff name (assim being abbreviated by A, P for production) 

For this, we will associate a **class decorator** with the predefined footprints attribute. This decorator will be applied by *footprints* at the time of the creation of the class. Here is the decorator function:

In [15]:
def _cutoffbis_deco(cls):
    def _get_abbrev_cutoff(self):
        return dict(assim='A', production='P').get(self.cutoff, 'X')
    cls.abbrev_cutoff = property(_get_abbrev_cutoff, doc="Abbreviated cutoff name.")
    return cls

The predefined *cutoff* attribute is associated with the newly created decorator: 

In [16]:
cutoff_deco = fp.DecorativeFootprint(cutoff,
                                     decorator = [_cutoffbis_deco, ])

Let's use this **DecorativeFootprint**:

In [18]:
class PerturedModelState(ModelState):
    
    _footprint = [cutoff_deco,
        dict(
            attr = dict(
                kind = dict(
                    values = ['perturbed_state', ]
                ),       
            )
        )]

The resulting class has an `abbrev_cutoff` *property* (added by the decorator): 

In [19]:
ps_obj = fpx.fakeresource(kind='perturbed_state', date='2017070100', cutoff='production')
print(ps_obj)
print('kind={0.kind:s}, date={0.date:s}, cutoff={0.cutoff:s}, abbrev_cutoff={0.abbrev_cutoff:s}'.format(ps_obj))

<__main__.PerturedModelState object at 0x7fbcd9e2b0f0 | footprint=3>
kind=perturbed_state, date=2017070100, cutoff=production, abbrev_cutoff=P


### Detailed description of the attributes

It is possible to use a large number of keys when describing attributes:

  * **type**: type (in the Python sense) of the attribute (*str* by default);
  * **outcast**: opposite of *values*: we specify the forbidden values and not those allowed;
  * **optional**: an attribute can be optional (by default an attribute is mandatory);
  * **default**: if ``optional=True`` we can specify a default value (*None* by default);
  * **alias**: during the creation phase of the objects (footprint resolution), the same attribute can have several synonyms;
  * **remap**: it is possible to point an allowed value to another;

In [20]:
class Historic(ModelState):
    
    _footprint = dict(
        attr = dict(
            kind = dict(
                values = ['historic', 'forecastfile', ],
                remap = dict(forecastfile='historic', ),
            ),
            term = dict(
                info = "The forecast's term",
                type = int,
                alias = ('fcterm', 'forecastterm', ),
            ),
            nativefmt = dict(
                info = "The storage's format.",
                values = ['grib', 'fa', 'netcdf', ],
                optional = True,
                default = 'grib',
            ),
        )
    )

#### Demonstration for *remap* and *optional*

Reminder:

  * for *kind*, ``remap = dict(forecastfile='historic', )`` ;
  * for *nativefmt*, ``optional = True`` and ``default = 'grib'``.

In [21]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            term=6)
print(obj_hst1, "object's kind:     ", obj_hst1.kind)
print(obj_hst1, "object's nativefmt:", obj_hst1.nativefmt)

<__main__.Historic object at 0x7fbcd9e327b8 | footprint=5> object's kind:      historic
<__main__.Historic object at 0x7fbcd9e327b8 | footprint=5> object's nativefmt: grib


#### Demonstration for *alias*

Reminder: for *term*, ``alias = ('fcterm', 'forecastterm', ),``.

In [22]:
obj_hst1 = fpx.fakeresource(kind='forecastfile',
                            cutoff='assim', date='20170701',
                            fcterm=6)
print(obj_hst1, "object's term:  ", obj_hst1.term)
try:
    print(obj_hst1, "object's fcterm:", obj_hst1.fcterm)
except AttributeError:
    print("Notice that 'fcterm' does not really exist.")

<__main__.Historic object at 0x7fbcd9e2b940 | footprint=5> object's term:   6
Notice that 'fcterm' does not really exist.


#### Demonstration for *type*

Reminder: for *term*, ``type = int,``

In [25]:
# Ok
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='6')
print("{0!r} object's term is {0.term:d}\n".format(obj_hst1))
# KO
print('This will fail:')
obj_hst1 = fpx.fakeresource(kind='forecastfile', cutoff='assim', date='20170701',
                            term='This is not an integer !')

# [2019/08/27-16:53:27][footprints.collectors][pickup_and_cache:0296][WARNING]: No fakeresource found in description 
      dict(
          cutoff = 'assim', 
          date = '20170701', 
          fakeresource = None, 
          kind = 'forecastfile', 
          term = 'This is not an integer !',
      )


<__main__.Historic object at 0x7fbcdfb2ed30> object's term is 6

This will fail:

 Report Footprint-Fakeresource: 

     __main__.Historic
         term       : {'args': ('int', 'This is not an integer !'), 'why': 'Could not reclass'}

     __main__.InitialConditions
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}

     __main__.Observations
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}

     __main__.PerturedModelState
         kind       : {'args': 'forecastfile', 'why': 'Not in values'}



Here we used a simple type that pre-existed in the Python language (*int*), it is of course possible to use custom types (the only constraint is that such a type must be “hashable”). 

This is what we should have been done for the *date* and *term* attributes. Indeed: 

    * A *data* is not a simple string of characters (it must be verified that the date provided is valid);
    * A *term* is unfortunately not an integer but can be expressed in hours and minutes. 
    
If necessary, the use of custom types is strongly recommended (see, for example, classes **bronx.stdtypes.date.Date** and **bronx.stdtypes.date.Time**).

## Notion of predefined values

It is often desirable to specify, at the beginning of the script, predefined values for a certain number of attributes. In our previous examples, we imagine pretty well that the user may want to set a *date* and a *cutoff*. 

It si enough to set these values through the **setup** object of the **footprints** package. 

NB: These are only predefined values: if the user specifies another value during the creation of the object, it will take precedence. 

Let's define default *cutoff* and *date*: 

In [26]:
fp.setup.defaults =  dict(cutoff='assim', date='20170701')

# Using the defaut:
obj_hst2a = fpx.fakeresource(kind='forecastfile', term=6)
print('{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2a))
# Manually redefine the cutoff value:
obj_hst2b = fpx.fakeresource(kind='forecastfile', cutoff='production', term=6)
print('{0!r}: cutoff={0.cutoff:s}, date={0.date:s}'.format(obj_hst2b))

<__main__.Historic object at 0x7fbcd9e93240>: cutoff=assim, date=20170701
<__main__.Historic object at 0x7fbcd9e93128>: cutoff=production, date=20170701


## Conclusion

When creating classes using **footprints**:

* To make good use of inheritance and to declare abstract classes as such; 
* For the most common attributes, define pre-defined footprints and use them; 
* In addition to pre-defined footprints, define class decorators to alter the class behaviour consistently with the pre-defined attribute(s);
* Please add “info” keys that are optional but appear in the automatic documentation; 
* Describe the types of data as precisely as possible.

Any questions: *vortex.support@meteo.fr*